In [1]:
import pandas as pd 
import numpy as np

In [2]:
df=pd.read_csv("C:/Users/galib/Downloads/food-price-index-september-2023-index-numbers.csv")

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,FunctionTransformer,OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
df=df.dropna(subset=['Data_value'])

In [5]:
df

,Series_reference,Period,Data_value,STATUS,UNITS,Subject,Group,Series_title_1
0,CPIM.SE901,1960.01,45.923461,FINAL,Index,Consumers Price Index - CPI,Food Price Index for New Zealand,Food
1,CPIM.SE901,1960.02,45.498637,FINAL,Index,Consumers Price Index - CPI,Food Price Index for New Zealand,Food
2,CPIM.SE901,1960.03,45.116296,FINAL,Index,Consumers Price Index - CPI,Food Price Index for New Zealand,Food
3,CPIM.SE901,1960.04,45.158779,FINAL,Index,Consumers Price Index - CPI,Food Price Index for New Zealand,Food
4,CPIM.SE901,1960.05,45.286226,FINAL,Index,Consumers Price Index - CPI,Food Price Index for New Zealand,Food
...,...,...,...,...,...,...,...,...
12863,CPIM.SE901502,2023.05,1347.000000,FINAL,Index,Consumers Price Index - CPI,Food Price Index Level 3 Classes for New Zealand,Ready-to-eat food
12864,CPIM.SE901502,2023.06,1361.000000,FINAL,Index,Consumers Price Index - CPI,Food Price Index Level 3 Classes for New Zealand,Ready-to-eat food
12865,CPIM.SE901502,2023.07,1366.000000,FINAL,Index,Consumers Price Index - CPI,Food Price Index Level 3 Classes for New Zealand,Ready-to-eat food
12866,CPIM.SE901502,2023.08,1367.000000,FINAL,Index,Consumers Price Index - CPI,Food Price Index Level 3 Classes for New Zealand,Ready-to-eat food


In [6]:
X=df.drop('Data_value',axis=1)
y=df['Data_value']

In [7]:
def parse_year_dot_month(X):
    if isinstance(X,pd.DataFrame):
        X=X.values
    years=np.floor(X).astype(int).flatten()
    months=np.round((X-years.reshape(-1,1))*100).astype(int).flatten()
    return pd.DataFrame({
        'year':years,
        'month':months
    })


In [8]:
columns_to_drop=['Series_reference', 'Series_title_1', 'UNITS', 'Subject']
categorica_cols=['STATUS','Group']
datetime_cols=['Period']


In [9]:
preprocessor=ColumnTransformer(transformers=[
    ('columns_to_drop','drop',columns_to_drop),
    ('cat',OrdinalEncoder(),categorica_cols),
    ('date',FunctionTransformer(parse_year_dot_month, validate=False),datetime_cols) 
])
pipeline=Pipeline(steps=[
    ('preprocessing',preprocessor),
    ('regressor',RandomForestRegressor(random_state=42))

])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [11]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [12]:
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R2 Score: {r2:.2f}")

MAE: 60.50
MSE: 7339.48
R2 Score: 0.88
